In [1]:
%pip install -Uq upgini
import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot as plt, style
style.use('seaborn-darkgrid')
import seaborn as sns
sns.set_style('darkgrid')
import plotly.express as px
from warnings import filterwarnings, simplefilter
filterwarnings('ignore')
simplefilter('ignore')

In [2]:
train_start = "2013-01-01"
train_end = "2017-08-15"
test_start = "2017-08-16"
test_end = "2017-08-31"

train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv',
                    parse_dates = ['date'], infer_datetime_format = True,
                    dtype = {'store_nbr' : 'category',
                             'family' : 'category'},
                    usecols = ['date', 'store_nbr', 'family', 'sales'])
train = train.set_index(['date', 'store_nbr', 'family']).sort_index()
print(train.shape)
train.head()

In [3]:
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv',
                   parse_dates = ['date'], infer_datetime_format = True)
test = test.set_index(['date', 'store_nbr', 'family']).sort_values('id')
print(test.shape)
test.head()

### oil特徴量の追加

In [4]:
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv',
                  parse_dates = ['date'], infer_datetime_format = True,
                  index_col = 'date')
oil['avg_oil'] = oil['dcoilwtico'].rolling(7).mean()
df_oil = pd.DataFrame(index=pd.date_range(train_start, test_end), columns=["dcoilwtico", "avg_oil"])
df_oil.loc[train_start:test_end, :] = oil.loc[train_start:test_end]
df_oil = df_oil.astype(np.float64).interpolate()
df_oil.tail()

In [5]:
import statsmodels.api as sm

st = "2013-01-10"
ed = "2017-08-15"

y_oil = df_oil.loc[st:ed]["dcoilwtico"]
data = train.groupby(["date", "family"]).mean()
familys = data.index.get_level_values("family").unique()
plot_size = 30

for family in familys:
    # salesデータはクリスマスが欠損してるので補完する
    a = pd.DataFrame(data.query(" family==@family ").sales.to_numpy(), index=pd.to_datetime(data.query(" family==@family ").index.get_level_values("date")), columns=["sales"])
    df_new = pd.DataFrame(index=pd.date_range(st, ed), columns=["sales"])
    df_new.loc[a.loc[st:ed].index, :] = a.loc[st:ed]
    df_new["sales"] = df_new["sales"].fillna(0) # クリスマスは売り上げ0にしておく
    y_family = df_new["sales"].to_numpy().reshape(-1, )
    ccf_xy = sm.tsa.ccf(y_family, y_oil)[1:plot_size+1]
    ccf_yx = sm.tsa.ccf(y_oil, y_family)[:plot_size]
    ccf = np.concatenate([ccf_yx[::-1], ccf_xy])
    
    x_axis = np.arange(-plot_size, plot_size)
    fig = plt.figure(figsize=(6, 3), dpi=120)
    ax = fig.add_subplot(111)
    ax.stem(x_axis, ccf)
    ax.set_xlim([-plot_size, plot_size])
    ax.set_ylim([-1, 1])
    ax.set_title(f"{family} vs Oil")
    plt.show()

In [6]:
add_f = pd.DataFrame(index=pd.date_range(train_start, test_end), columns=["dcoilwtico", "avg_oil"])
add_f.loc[train_start:test_end, :] = df_oil.loc[train_start:test_end]
add_f.head()

### 曜日の追加

In [7]:
dow = pd.date_range(train_start, test_end).dayofweek + 1
add_f["dayofweek_cos"] = np.cos(2 * np.pi * dow / dow.max())
add_f["dayofweek_sin"] = np.sin(2 * np.pi * dow / dow.max())
add_f.head(10)

### 祝日の追加

In [8]:
hol = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv',
                  parse_dates = ['date'], infer_datetime_format = True, index_col = 'date')
hol = hol[hol.locale == 'National']  # National holidayだけ使う
hol = hol.groupby(hol.index).first() # 重複してる日付は消す
add_f = pd.concat([add_f, hol], axis=1).loc[train_start:test_end] 
add_f["wd"] = 1 # Work Day
add_f.loc[add_f.index.dayofweek > 4, "wd"] = 0 # 土日はWork Dayじゃない
add_f.loc[add_f.type=="Work Day", "wd"] = 1 # Work Day
add_f.loc[add_f.type=="Transfer", "wd"] = 0 # 祝日転送?
add_f.loc[add_f.type=="Bridge", "wd"] = 0
add_f.loc[(add_f.type == 'Holiday') & (add_f.transferred == False), 'wd'] = 0 # 祝日で転送されてない
add_f.loc[(add_f.type == 'Holiday') & (add_f.transferred == True), 'wd'] = 1 # 祝日だけど別日に転送されたのでWork Day
add_f = pd.get_dummies(add_f, columns = ['type']) # 
add_f.drop(['locale', 'locale_name', 'description', 'transferred'], axis=1, inplace=True) # Unused columns

add_f.head()

### 給料日

In [9]:
add_f["wageday"] = ((add_f.index.is_month_end) | (add_f.index.day==15)).astype(int)

add_f.head()

### school season

In [10]:
school_season = []
for i, r in add_f.iterrows():
    if i.month in [4, 5, 8, 9] :
        school_season.append(1)
    else :
        school_season.append(0)
add_f['school_season'] = school_season
add_f.head()

In [19]:
add_f.to_csv("train_add_f.csv", index=None)